In [2]:
import copy
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from modules import solver, presolver
from pyballistics import ozvb_lagrange, get_powder_names
from tqdm import tqdm

## Нахождение начальных точек
формируется таблица с начальными точками для каждого пороха

In [3]:
tabel_init_points = pd.read_csv("data/init_points.csv")
tabel_init_points.loc[tabel_init_points['success']==False]

,dbname,wq,ro,reason,success


## Решение обратной задачи

In [4]:
tabel_result = list()

In [ ]:
%%time

for _, row in tqdm(tabel_init_points.iterrows()):
    initial_dict = presolver.init_dict(row['dbname'], row['wq'], row['ro'])
    #print("name: ", row['dbname'], " wq: ", row['wq'], " ro ", row['ro'])
    solv = solver.Solver(initial_dict)
    solv.solution()
    tabel_result.append(solv.get_solution())
        

0it [00:00, ?it/s]

Optimization terminated successfully.
         Current function value: 46377.918725
         Iterations: 105
         Function evaluations: 200
Optimization terminated successfully.
         Current function value: 46377.918725
         Iterations: 93
         Function evaluations: 174
No success in result
Optimization terminated successfully.
         Current function value: 992.757071
         Iterations: 87
         Function evaluations: 168
Optimization terminated successfully.
         Current function value: 928.111222
         Iterations: 108
         Function evaluations: 206
Optimization terminated successfully.
         Current function value: 8367.414202
         Iterations: 74
         Function evaluations: 147
Optimization terminated successfully.
         Current function value: 859.845700
         Iterations: 106
         Function evaluations: 204
Optimization terminated successfully.
         Current function value: 1264.952612
         Iterations: 121
         Function

E:\PROGRAMMING PROJECT\Jupyter project\BMSTU\5th year\Gun Projecting\modules\solver.py:223: RuntimeWarning: divide by zero encountered in true_divide
  self.n_real = self.pressure_tube/self.pressure


Optimization terminated successfully.
         Current function value: 19869.114167
         Iterations: 65
         Function evaluations: 132
Optimization terminated successfully.
         Current function value: 855.711869
         Iterations: 103
         Function evaluations: 191
Optimization terminated successfully.
         Current function value: 723.142407
         Iterations: 70
         Function evaluations: 145
Optimization terminated successfully.
         Current function value: 652.738360
         Iterations: 73
         Function evaluations: 137
Optimization terminated successfully.
         Current function value: 747.325416
         Iterations: 86
         Function evaluations: 159
Optimization terminated successfully.
         Current function value: 759.903356
         Iterations: 84
         Function evaluations: 162
Optimization terminated successfully.
         Current function value: 679.419270
         Iterations: 150
         Function evaluations: 308
Optimizat

In [5]:
result = copy.deepcopy(tabel_result)

In [ ]:
table = pd.DataFrame(result)
#table.to_csv("data/table_result.csv", index=False)

## Обработка

In [ ]:
table.head()